# **Step-by-Step Guide: Installing PyTorch with CUDA on NVIDIA Jetson**

Installation on NVIDIA Jetson devices (Nano, Xavier, Orin) is different from standard x86 desktops because the Jetson uses an ARM architecture and relies on the specialized JetPack SDK. You cannot use the standard `pip install` commands from PyTorch.org.

### **Prerequisites**

**1. NVIDIA Jetson Device:** Running the latest version of the JetPack SDK.

**2. Internet Connection:** Required for downloading packages.

**3. Terminal Access:** Via SSH or directly on the device.

## **Step 1: Determine Your JetPack Version**

The most critical step is identifying your running JetPack version, as the PyTorch wheel file (`.whl`) must be built specifically for it.

**1. Open a terminal on your Jetson.**

**2. Run the following command to check your operating system information, which usually includes the JetPack version (e.g., 4.6.1, 5.1.2):**

`cat /etc/nv_tegra_release`


*Note the major version (e.g., 5.x or 6.x).*

## **Step 2: Locate the Official NVIDIA PyTorch Wheel (.whl)**

NVIDIA provides custom PyTorch builds for the Jetson architecture. You must find the wheel that matches your JetPack version.

Navigate to the official **NVIDIA Developer Forum for PyTorch on Jetson**. (A quick search for "PyTorch Jetson" will usually lead you to the correct thread.)

In that thread, look for the most recent post that provides download links. It will specify which PyTorch version works with which JetPack version.

Find the `wget` link for the PyTorch and, optionally, the TorchVision .whl file that corresponds to your system.

## **Step 3: Install Required Dependencies**

Before installing PyTorch, install required system dependencies:


`sudo apt update`


`sudo apt install python3-pip libopenblas-base libjpeg-dev zlib1g-dev`


## **Step 4: Download and Install the PyTorch Wheel**

Use the `wget` link you found in Step 2 to download the wheel file to your Jetson, and then install it using `pip3`.

**1. Download the PyTorch Wheel:**

`# Replace [URL_TO_PYTORCH.whl] with the link you found`


`wget [URL_TO_PYTORCH.whl]`


**2. Install the PyTorch Wheel:**

`# Replace [FILENAME.whl] with the downloaded file's name`


`pip3 install [FILENAME.whl]`


## **Step 5: Install TorchVision**

If you need TorchVision (we do need it for MobileNetV2), you often need to build it from source or use a pre-compiled wheel due to architecture differences.

**1. Install dependencies for TorchVision:**

`sudo apt install libxml2-dev libxslt1-dev libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev`


**2. Download and Install TorchVision: (Again, check the NVIDIA forum for the correct TorchVision .whl file link for your JetPack version and repeat Step 4 using those links.)**

## **Step 6: Verification**
Finally, verify the installation by running Python and checking CUDA availability:




In [1]:
import torch
import sys

def check_cuda_setup():
    """Checks the PyTorch and CUDA installation status."""
    
    print("--- PyTorch and CUDA Setup Diagnosis ---")
    print(f"PyTorch Version: {torch.__version__}")
    print(f"Python Version: {sys.version.split()[0]}")
    print("-" * 40)
    
    # Check 1: Is PyTorch built with CUDA support?
    # This checks which CUDA version PyTorch was compiled against.
    cuda_build_version = torch.version.cuda
    print(f"PyTorch built with CUDA version: {cuda_build_version}")
    
    # Check 2: Is CUDA currently available for use?
    cuda_is_available = torch.cuda.is_available()
    print(f"torch.cuda.is_available(): {cuda_is_available}")

    if not cuda_is_available:
        print("\n\t*** DIAGNOSIS: CUDA is NOT available to PyTorch. ***")
        print("\tCheck your PyTorch installation (did you use the CUDA version?)")
        print("\tCheck your NVIDIA drivers and CUDA Toolkit installation.")
        return

    # If CUDA is available, check GPU details
    print("-" * 40)
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPUs detected: {gpu_count}")
    
    if gpu_count > 0:
        for i in range(gpu_count):
            gpu_name = torch.cuda.get_device_name(i)
            print(f"GPU {i} Name: {gpu_name}")
            print(f"GPU {i} Capability: {torch.cuda.get_device_capability(i)}")
            print(f"Total Memory: {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GB")
    
    # Check 3: CuDNN status (often used for performance)
    cudnn_is_available = torch.backends.cudnn.is_available()
    print("-" * 40)
    print(f"torch.backends.cudnn.is_available(): {cudnn_is_available}")

    if cuda_is_available and gpu_count > 0:
        # Final check: can we move a tensor to the device?
        try:
            test_tensor = torch.randn(2, 2).to('cuda:0')
            print(f"Test tensor successfully moved to device: {test_tensor.device}")
        except Exception as e:
            print(f"Failed to move test tensor to GPU: {e}")

if __name__ == '__main__':
    check_cuda_setup()

--- PyTorch and CUDA Setup Diagnosis ---
PyTorch Version: 2.0.1+cu117
Python Version: 3.9.13
----------------------------------------
PyTorch built with CUDA version: 11.7
torch.cuda.is_available(): True
----------------------------------------
Number of GPUs detected: 1
GPU 0 Name: NVIDIA GeForce RTX 3060 Laptop GPU
GPU 0 Capability: (8, 6)
Total Memory: 6.00 GB
----------------------------------------
torch.backends.cudnn.is_available(): True
Test tensor successfully moved to device: cuda:0


Once this check passes, your Jetson is properly configured for GPU-accelerated PyTorch workloads.